In [1]:
import folium
import clustering as cl
import networkx as nx
import spatial as sp
import matplotlib.pyplot as plt

In [2]:
metro = nx.read_gml("data/graph/metro.gml")
metro_nodes = list(metro.nodes(data = True))
stops = metro_nodes# + bus_nodes + tram_nodes
regions = cl.cluster_stops(stops, 1)

In [3]:
points = [x["coord"] for _, x in regions.items()]
region_neighbors = cl.get_neighbor_list(points)

In [4]:
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([max(points[0]), max(points[1])], [min(points[0]), min(points[1])])

In [6]:
# Weird vertices in voronoi_finite, might not work correctly
sp_regions, sp_vertices = sp.voronoi_finite(points)

# Missing: drawing voronoi over map

In [15]:
region_neighbors = cl.get_neighbor_list(points)
neighbor_graph = nx.Graph(region_neighbors)

for n in neighbor_graph:
    neighbor_graph.nodes[n]["coord"] = regions[n]["coord"]
    
    folium.Circle(
        radius = 100,
        location = neighbor_graph.nodes[n]["coord"],
        popup = n,
        color = '#3186cc',
        fill = True,
        fill_color='#3186cc'
    ).add_to(m)

In [19]:
m